In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Oturum ve başlık ayarları
session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://www.tjk.org/TR/YarisSever/Query/Page/KosuSorgulama',
    'X-Requested-With': 'XMLHttpRequest'
}

# Ana sayfayı çekerek gerekli token'ları al
print("Siteye bağlanıyor...")
initial_response = session.get('https://www.tjk.org/TR/YarisSever/Query/Page/KosuSorgulama', headers=headers)
initial_soup = BeautifulSoup(initial_response.text, 'html.parser')

# Anti-forgery token kontrolü
token = initial_soup.find('input', {'name': '__RequestVerificationToken'})
token_value = token['value'] if token else ''
print("Token alındı." if token else "Token bulunamadı, devam ediliyor...")

# İlk sayfadan verileri çek
print("İlk sayfa verileri çekiliyor...")
all_rows = []
table = initial_soup.find('table', {'class': 'table table-striped'}) or initial_soup.find('table')
if table:
    all_rows += table.find_all('tr')

# Sonraki sayfaları çek
for page_num in range(2, 14):  # 2'den 13'e kadar (13 sayfa)
    print(f"{page_num}. sayfa çekiliyor...")
    page_url = f"https://www.tjk.org/TR/YarisSever/Query/DataRows/KosuSorgulama?PageNumber={page_num}&Sort=Tarih+desc%2C+Sehir+asc%2C+KosuSirasi+asc"
    response = session.get(page_url, headers=headers)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    rows = soup.find_all('tr')
    all_rows += rows
    time.sleep(0.5)  # Sunucuyu üzmemek için ufak bir bekleme

# Sütun başlıkları
columns = ['Tarih', 'Şehir', 'Koşu', 'Grup', 'Koşu Cinsi', 'Apr. Koş. Cinsi', 
           'Mesafe', 'Pist', 'Sıklet', 'Orijin (Baba-Anne)', 'İkramiye', 
           'Birinci', 'Yaş', 'Derece', 'H. Puanı']

# Verileri parse et
print("Veriler işleniyor...")
races = []
for row in all_rows:
    cols = row.find_all(['th', 'td'])
    if cols:
        race_data = {}
        for i, header in enumerate(columns):
            if i < len(cols):
                race_data[header] = cols[i].text.strip()
            else:
                race_data[header] = ''
        races.append(race_data)

# DataFrame oluştur
df = pd.DataFrame(races)

# Temizlik: Boş satırları kaldır
df = df[df['Tarih'].str.len() > 0]

# CSV olarak kaydet
csv_filename = 'tjk_yarislar_2025_full.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
print(f"\nBaşarıyla kaydedildi: {csv_filename}")
print(f"Toplam {len(df)} yarış verisi bulundu.")

# Örnek verileri göster
print("\nİlk 3 yarış bilgisi:")
print(df.head(3).to_string(index=False))

Siteye bağlanıyor...
Token bulunamadı, devam ediliyor...
İlk sayfa verileri çekiliyor...
2. sayfa çekiliyor...
3. sayfa çekiliyor...
4. sayfa çekiliyor...
5. sayfa çekiliyor...
6. sayfa çekiliyor...
7. sayfa çekiliyor...
8. sayfa çekiliyor...
9. sayfa çekiliyor...
10. sayfa çekiliyor...
11. sayfa çekiliyor...
12. sayfa çekiliyor...
13. sayfa çekiliyor...
Veriler işleniyor...

Başarıyla kaydedildi: tjk_yarislar_2025_full.csv
Toplam 664 yarış verisi bulundu.

İlk 3 yarış bilgisi:
     Tarih  Şehir Koşu                Grup Koşu Cinsi Apr. Koş. Cinsi Mesafe Pist Sıklet                              Orijin (Baba-Anne) İkramiye          Birinci     Yaş  Derece H. Puanı
     Tarih  Şehir Koşu                Grup Koşu Cinsi Apr. Koş. Cinsi Mesafe Pist Sıklet                              Orijin (Baba-Anne) İkramiye          Birinci     Yaş  Derece H. Puanı
26.04.2025 Ankara    1  3 Yaşlı İngilizler     Maiden                   1400  Kum     58 RUN AWAY AND HIDE (USA) - PERFECT HOLIDAY (USA)  430

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Oturum ve başlık ayarları
session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}

# Hepsini buradan çekelim: 1'den 13'e
all_rows = []

for page_num in range(1, 14):  # 1. sayfa dahil
    print(f"{page_num}. sayfa çekiliyor...")
    page_url = f"https://www.tjk.org/TR/YarisSever/Query/DataRows/KosuSorgulama?PageNumber={page_num}&Sort=Tarih+desc%2C+Sehir+asc%2C+KosuSirasi+asc"
    response = session.get(page_url, headers=headers)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    rows = soup.find_all('tr')
    all_rows += rows
    time.sleep(0.5)  # Sunucuyu üzmemek için

# Sütun başlıkları
columns = ['Tarih', 'Şehir', 'Koşu', 'Grup', 'Koşu Cinsi', 'Apr. Koş. Cinsi', 
           'Mesafe', 'Pist', 'Sıklet', 'Orijin (Baba-Anne)', 'İkramiye', 
           'Birinci', 'Yaş', 'Derece', 'H. Puanı']

# Verileri parse et
print("Veriler işleniyor...")
races = []
for row in all_rows:
    cols = row.find_all(['td'])
    if cols:
        race_data = {}
        for i, header in enumerate(columns):
            if i < len(cols):
                race_data[header] = cols[i].text.strip()
            else:
                race_data[header] = ''
        races.append(race_data)

# DataFrame oluştur
df = pd.DataFrame(races)

# Temizlik: Boş satırları kaldır
df = df[df['Tarih'].str.len() > 0]

# CSV olarak kaydet
csv_filename = 'tjk_yarislar_2025_full.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
print(f"\nBaşarıyla kaydedildi: {csv_filename}")
print(f"Toplam {len(df)} yarış verisi bulundu.")

# Örnek verileri göster
print("\nİlk 3 yarış bilgisi:")
print(df.head(3).to_string(index=False))

1. sayfa çekiliyor...
2. sayfa çekiliyor...
3. sayfa çekiliyor...
4. sayfa çekiliyor...
5. sayfa çekiliyor...
6. sayfa çekiliyor...
7. sayfa çekiliyor...
8. sayfa çekiliyor...
9. sayfa çekiliyor...
10. sayfa çekiliyor...
11. sayfa çekiliyor...
12. sayfa çekiliyor...
13. sayfa çekiliyor...
Veriler işleniyor...

Başarıyla kaydedildi: tjk_yarislar_2025_full.csv
Toplam 663 yarış verisi bulundu.

İlk 3 yarış bilgisi:
     Tarih  Şehir Koşu                   Grup                Koşu Cinsi Apr. Koş. Cinsi Mesafe Pist Sıklet                       Orijin (Baba-Anne) İkramiye      Birinci     Yaş  Derece H. Puanı
24.04.2025 Ankara    7     3 Yaşlı İngilizler                      KV-8                   1500  Kum     58                        TOUGH GUY - AMARİ  810.000     GÖK BERK 3y a  e 1.33.59       91
24.04.2025 Ankara    8 3 ve Yukarı İngilizler Kısa Vade Handikap 24 /H1                   1500  Kum   54,5      SIDNEY'S CANDY (USA) - MARLA SINGER  810.000 MADAM ORACLE  5y d k 1.33.81       90